In [1]:
import requests

url = "https://ftp.ncbi.nlm.nih.gov/pubmed/baseline/"
resp = requests.get(url)

In [3]:
links = BeautifulSoup(resp.content, parse_only=SoupStrainer('a'))

In [4]:
links = list(links)

In [5]:
links[1].has_attr

<bound method Tag.has_attr of <a href="/pubmed/">Parent Directory</a>>

In [ ]:
from os.path import exists
from tqdm import tqdm
import os

datadir = "/home/jovyan/data"
if not exists(datadir):
    os.makedirs(datadir)

for link in tqdm(links):
    if hasattr(link, 'has_attr'):
        if link.has_attr('href'):
            href = link['href']
            if href.startswith('pubmed23') and href.endswith('.xml.gz'):
                file_url = f"{url}{href}"
                target = f"{datadir}/{href}"
                if exists(target):
                    continue
                with open(target, "wb+") as f:
                    f.write(requests.get(file_url).content)


NameError: name 'links' is not defined

In [4]:
import gzip
import lxml.etree as etree
with gzip.open("/home/jovyan/data/pubmed23n1149.xml.gz") as f:
    tree = etree.parse(f).getroot()

In [5]:
nodes = list(tree)

In [9]:
node = nodes[0]
print(etree.tostring(node).decode('utf-8'))

<PubmedArticle>
    <MedlineCitation Status="MEDLINE" IndexingMethod="Automated" Owner="NLM">
      <PMID Version="1">35949070</PMID>
      <DateCompleted>
        <Year>2022</Year>
        <Month>08</Month>
        <Day>12</Day>
      </DateCompleted>
      <DateRevised>
        <Year>2022</Year>
        <Month>08</Month>
        <Day>12</Day>
      </DateRevised>
      <Article PubModel="Print-Electronic">
        <Journal>
          <ISSN IssnType="Electronic">1879-2448</ISSN>
          <JournalIssue CitedMedium="Internet">
            <Volume>221</Volume>
            <PubDate>
              <Year>2022</Year>
              <Month>Aug</Month>
              <Day>01</Day>
            </PubDate>
          </JournalIssue>
          <Title>Water research</Title>
          <ISOAbbreviation>Water Res</ISOAbbreviation>
        </Journal>
        <ArticleTitle>Evaluating the role of biochar in mitigating the inhibition of polyethylene nanoplastics on anaerobic granular sludge.</ArticleTitle>


In [15]:

def to_string(x) -> str:
    if isinstance(x, bytes):
        return x.decode("utf-8")
    return x


In [16]:
def parse(node):
    text = None
    source = None
    title = None
    _ = node.find('.//AbstractText')
    if _ is not None:
        text = _.text
        text = to_string(text)
    _ = node.find('.//ArticleTitle')
    if _ is not None:
        title = _.text
        title = to_string(title)
    _ = node.find('.//ArticleTitle')
    if _ is not None:
        source = etree.tostring(_).decode('utf-8')
        source = to_string(source)
    journal_title_node = node.find('.//Journal/Title')
    if journal_title_node is not None:
        publication_title = journal_title_node.text
    return dict(text=text, source=source, title=title, publication_title=publication_title)


In [17]:
from tqdm.contrib.concurrent import process_map

In [18]:
from os.path import join, exists
import gzip
import lxml.etree as etree
from tqdm import tqdm
import os
import json
import tempfile
import shutil

datadir = "/home/jovyan/data"
output = "/home/jovyan/json-raw"
tmpdir = "/home/jovyan/tmp"


if not exists(output):
    os.makedirs(output)
if not exists(tmpdir):
    os.makedirs(tmpdir)
    
    
def doit(datadir, output, fname):
    output_path = join(output, fname) + ".jsonl"
    if exists(output_path):
        return
    ff = join(datadir, fname)
    with gzip.open(ff) as f:
        # data = f.read()
        tree = etree.parse(f).getroot()
        all_data = []
        for node in tree:
            data = parse(node)
            all_data.append(data)
    try:
        with tempfile.NamedTemporaryFile(mode='w+', dir=tmpdir, delete=False) as fout:        
            for data in all_data:
                if data['text'] and data['title']:    
                    fout.write(json.dumps(data))
                    fout.write('\n')
            shutil.move(fout.name, output_path)
    except Exception as e:
        raise
    finally:
        if exists(fout.name):
            os.remove(fout.name)
            
def partial(fname):
    return doit(datadir, output, fname)
# for f in tqdm(os.listdir(datadir)):
#     doit(datadir, output, f)
    
process_map(partial, os.listdir(datadir), max_workers=8)

/tmp/ipykernel_14583/1717205241.py:51: TqdmWarning: Iterable length 1166 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  process_map(partial, os.listdir(datadir), max_workers=8)


  0%|          | 0/1166 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
data

In [96]:
node = list(tree)[300]

In [102]:
node.__class__

lxml.etree._Element

b'<ArticleTitle>Usefulness of an elevated B-type natriuretic peptide in predicting survival in patients with aortic stenosis treated without surgery.</ArticleTitle>\n        '

In [82]:
print(etree.tostring(node, pretty_print=True).decode('utf-8'))

<PubmedArticle>
    <MedlineCitation Status="MEDLINE" Owner="NLM">
      <PMID Version="1">16274891</PMID>
      <DateCompleted>
        <Year>2006</Year>
        <Month>02</Month>
        <Day>16</Day>
      </DateCompleted>
      <DateRevised>
        <Year>2016</Year>
        <Month>11</Month>
        <Day>24</Day>
      </DateRevised>
      <Article PubModel="Print">
        <Journal>
          <ISSN IssnType="Print">0899-7071</ISSN>
          <JournalIssue CitedMedium="Print">
            <Volume>29</Volume>
            <Issue>6</Issue>
            <PubDate>
              <MedlineDate>2005 Nov-Dec</MedlineDate>
            </PubDate>
          </JournalIssue>
          <Title>Clinical imaging</Title>
          <ISOAbbreviation>Clin Imaging</ISOAbbreviation>
        </Journal>
        <ArticleTitle>Concordance of mammographic classifications of microcalcifications in breast cancer diagnosis: Utility of the Breast Imaging Reporting and Data System (fourth edition).</ArticleTitle>
  